<a href="https://colab.research.google.com/github/paul-kim88/pythonProject2/blob/master/scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#!pip install lxml[html_clean] readability-lxml

import requests
from bs4 import BeautifulSoup
from readability import Document
import pandas as pd
import re

def duckduckgo_scrape(query, time='d'):
    url = f"https://duckduckgo.com/html/?q={query}&t=h_&df={time}&iar=news&ia=news"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes

        soup = BeautifulSoup(response.content, "html.parser")

        results = []
        for result in soup.find_all("div", class_="result"):
            title_element = result.find("a", class_="result__a")
            if title_element:
                title = title_element.text
                link = title_element["href"]
                results.append({"title": title, "link": link})

        return results
    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def extract(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[\t\n]+', '\n', text)
    return text



if __name__ == "__main__":
    search_query = input("Type What you want : ")
    search_results = duckduckgo_scrape(search_query, '3d')

    results = []

    if search_results:
        print(f"Search Results for '{search_query}':")
        for result in search_results:
            if 'wiki' in result['link']:
              continue
            print(f"Title: {result['title']}")
            print(f"Link: {result['link']}")
            print("-" * 20)

            # Try to get the link, handling HTTP errors
            try:
                response = requests.get(result['link'])
                response.raise_for_status()  # Raise an exception for bad status codes

                soup = BeautifulSoup(response.content, "html.parser")
                doc = Document(response.content)
                content = extract(doc.summary())
                results.append({"title": result['title'], "link": result['link'], "content": content})

            except requests.exceptions.HTTPError as e:
                print(f"HTTP Error fetching {result['link']}: {e}")
            except Exception as e:
                print(f"Error fetching or processing {result['link']}: {e}")
    search_df=pd.DataFrame(results)
    search_df

Type What you want : 한덕수
Search Results for '한덕수':
Title: 한덕수 먼저? 尹 먼저?... "탄핵 쟁점 맞물려 같은 날 선고 가능성도"
Link: https://www.hankookilbo.com/News/Read/A2025030413340002178
--------------------
Title: 총리 한덕수 프로필 정보 - 나이 재산 고향 학력 공직 경력 Sns 등
Link: https://sensepeople.tistory.com/entry/%EC%B4%9D%EB%A6%AC-%ED%95%9C%EB%8D%95%EC%88%98-%ED%94%84%EB%A1%9C%ED%95%84-%EC%A0%95%EB%B3%B4
--------------------
Title: 한덕수 복귀 가능성 구실로…최상목, 마은혁 임명 '시간끌기' - 경향신문
Link: https://www.khan.co.kr/article/202503042021015
--------------------
HTTP Error fetching https://www.khan.co.kr/article/202503042021015: 403 Client Error: Forbidden for url: https://www.khan.co.kr/article/202503042021015
Title: 한덕수, 이번주 복귀할 듯...尹 탄핵심판 전 마은혁 임명 물건너가
Link: https://www.newspim.com/news/view/20250305000013
--------------------
Title: 한덕수 총리, 대통령 권한 어디까지 승계받나 - Bbc
Link: https://www.bbc.com/korean/articles/cp83d8em6q2o
--------------------
Title: 국무총리 한덕수 프로필 / 나이 / 고향 / 학력 / 가족 / 자녀 / 정치 경력
Link: https://perfectinfo.co.kr/entry/%EA%B5%A

In [11]:
pd.DataFrame(results)


,title,link,content
0,Ukraine's Zelenskyy says end of war with Russi...,https://apnews.com/article/russia-ukraine-war-...,\n \n ...
1,Ukraine war latest: Zelensky ready to work und...,https://www.bbc.com/news/live/c981p3dxnent,Zelensky extends olive branch to Trump after U...
2,Ukraine war latest: Trump warns Zelenskyy but ...,https://news.sky.com/story/trump-putin-zelensk...,\n Trump's ...
3,"Ukraine confronts a future without America, an...",https://www.economist.com/europe/2025/03/01/uk...,THE NEWS from Washington reached Ukraine in re...
4,"Ukraine | History, Flag, Population, President...",https://www.britannica.com/place/Ukraine,"Ukraine, country located in eastern Europe, t..."
5,U.S. officials don't know what will happen nex...,https://www.nbcnews.com/politics/trump-adminis...,WASHINGTON — After Friday’s public clash betwe...
6,Ukraine | Today's latest from Al Jazeera,https://www.aljazeera.com/where/ukraine/,"Scott Lucas, a Birmingham professor, examines ..."
